In [1]:
import random

def random_nonzero_elt(F):
    e = 0
    while e == 0:
        e = F.random_element()
    return e

# q-ary noisy channel producing t errors
def channel_t_errors(y, t):
    ylen = y.length()
    K = y.base_ring()
    err_pos = Subsets(ylen, t).random_element()
    return vector(K, [y[i] + random_nonzero_elt(K) if i in err_pos else y[i] for i in range(ylen)])

In [410]:
class ReedSolomon:
    def __init__(self, q, n, k):
        self.q = q
        self.n = n
        self.k = k
        
        K.<g> = GF(q)
        self.K = K
        self.g = g
        
        R_x.<x> = K[]
        self.R_x = R_x
        self.x = x
        
        R_XY.<X, Y> = K[]
        self.R_XY = R_XY
        self.X = X
        self.Y = Y
        
        self.t = floor((n - k) / 2)
        
    def encode(self, m):
        p = self.R_x(m.list())
        return vector(self.K, [p(self.g^i) for i in range(self.n)])
    
    def BerlekampWelch(self, u):
        # n contraintes linéaires
        # (deg A + 1) + (deg B + 1) variables = (n - t) + t + 1 = n + 1 variables
        M = matrix(self.K, self.n, self.n + 1)
        
        for i in range(self.n):  # n equations linéaires
            x_i = self.g^i
            for j in range(self.t + 1): # u_i A(x_i)
                M[i, j] = u[i] * x_i^j
            for j in range(self.n - self.t): # -B(x_i)
                M[i, self.t + 1 + j] = -x_i^j
                
        coeffs = M.right_kernel().basis_matrix().rows() # résolution du système d'equations linéaires
        if coeffs == []:
            return None
        coeffs = coeffs[0]
        
        A = self.R_x(coeffs[:self.t + 1].list())
        B = self.R_x(coeffs[self.t + 1:self.n + 1].list())
        root = (B // A).list()
        
        while len(root) < self.k:
            root.append(0)
        return vector(self.K, root)
    
    def __degQ_j_Sudan(self, r, j):
        return self.n - r - j * (self.k - 1) - 1
    
    def __makeM_Sudan(self, u, r, l):
        lenQ_j = lambda j: 0 if self.__degQ_j_Sudan(r, j)+1 < 0 else self.__degQ_j_Sudan(r, j)+1
        ncol = sum([lenQ_j(j) for j in range(l + 1)]) #(l + 1)*(self.n/l + 1)
        
        M = matrix(self.K, self.n, ncol)
        for i in range(self.n): # n equations
            x_i = self.g^i
            offset = 0
            for j in range(l + 1): # l + 1 polynômes Q_j
                degQ_j = self.__degQ_j_Sudan(r, j)
                if degQ_j < 0:
                    continue
                for k in range(degQ_j + 1):
                    M[i, offset + k] = u[i]^j * x_i^k
                offset += degQ_j + 1
        return M
    
    def __hamming_dist(self, x, y):
        y = y[:self.k]
        x = vector(self.K, x.list())
        return reduce(lambda a, b: a + 1 if b != 0 else a, (x - y).list(), 0)
    
    def __argmin(self, L, received_codeword):
        argmin = L[0]
        dmin = self.__hamming_dist(argmin, received_codeword)
        for i in range(1, len(L)):
            min_i = self.__hamming_dist(L[i], received_codeword)
            if min_i < dmin:
                dmin = min_i
                argmin = L[i]
        return argmin
    
    def __pad_with_zeroes(self, r):
        r = r.list()
        while len(r) < self.k:
            r.append(0)
        return vector(self.K, r)
    
    def Sudan(self, u, r, l):
        print("u=",u)
        assert(l > 0)
        assert(r >= self.t)
        
        M = self.__makeM_Sudan(u, r, l)
        print("M=", M)
        
        coeffs = M.right_kernel().basis_matrix().rows() # résolution du système d'equations linéaires
        if coeffs == []:
            return None
        coeffs = coeffs[0]
        print("coeffs=",  coeffs)
        
        Q = 0
        offset = 0
        for j in range(l + 1):
            degQ_j = self.__degQ_j_Sudan(r, j) + 1
            v = coeffs[offset:offset + degQ_j]
            Q += sum([self.X^i * v[i] for i in range(len(v))]) * self.Y^j
            offset += degQ_j
                
        print("Q(Y)=", Q.polynomial(self.Y))
        
        roots = Q.polynomial(self.Y).roots()
        if roots == []:
            return None
        
        roots = list(map(lambda x: self.__pad_with_zeroes(x[0]), roots))
        
        # maximum de vraissemblance
        return vector(self.K, self.__argmin(roots, u).list())
    
    def __Q_uv(u,v,n,p,a):
        Q_uv = 0
        for i in range(n):
            for j in range(p):
                Q_uv += binomial(i, u) * binomial(j, v) * a[i, j] * self.X^(i - u) * self.Y^(j - v)
        return Q_uv
    
    def __degQ_j_GS(self, r, j, m):
        return m * (self.n - r) - j * (self.k - 1) - 1
    
    def __makeM_GS(self, noisy_codeword, r, l, m):
        # Comme Sudan, n contraintes linéaires Q(x_i, u_i)=0,
        # auquel on ajoute que Q(x+x_i,Y+y_i) n'a pas de monômes de degré < m (*)
        # Le nb de contraintes implicites pour (*) est 2 parmi m+1
        # Donc au total on a n * (2 parmi m+1) eqs linéaires
        lenQ_j = lambda j: 0 if self.__degQ_j_GS(r, j, m)+1 < 0 else self.__degQ_j_GS(r, j, m)+1
        ncol = sum([lenQ_j(j) for j in range(l + 1)]) # (l + 1)*(self.n/l + 1)
        neqs = self.n * m * (m+1) / 2
        
        M = matrix(self.K, neqs, ncol)
        c = 0 # indice de la ligne courante
        for i in range(self.n):
            x_i = self.g^i
            for u in range(m):
                for v in range(m):
                    if u + v >= m:
                        continue
                    offset = 0
                    for j in range(l + 1): # l + 1 polynômes Q_j
                        degQ_j = self.__degQ_j_GS(r, j, m)
                        if degQ_j < 0: # TODO: utiliser while
                            continue
                        for k in range(degQ_j + 1):
                            col_idx = offset + k
                            M[c, col_idx] = binomial(col_idx, u) * binomial(j, v) * x_i^(col_idx - u) * noisy_codeword[i]^(j - v)
                        offset += degQ_j + 1
                    c += 1
        return M
                                      
    def GuruswamiSudan(self, u, m, l, r):
        assert(m > 1)
        assert(l > 0)
        assert(r >= self.t)
        
        M = self.__makeM_GS(u, r, l, m)
        print("M=", M)
        
        coeffs = M.right_kernel().basis_matrix().rows() # résolution du système d'equations linéaires
        if coeffs == []:
            return None
        coeffs = coeffs[0]
        print("coeffs=",  coeffs)
        
        Q = 0
        offset = 0
        for j in range(l + 1):
            degQ_j = self.__degQ_j_GS(r, j, m) + 1
            v = coeffs[offset:offset + degQ_j]
            Q += sum([self.X^i * v[i] for i in range(degQ_j)]) * self.Y^j
            offset += degQ_j
            
        print("Q(Y)=", Q.polynomial(self.Y))
        roots = Q.polynomial(self.Y).roots()
        if roots == []:
            return None
                
        roots = list(map(lambda x: self.__pad_with_zeroes(x[0]), roots))
        
        # maximum de vraissemblance
        return vector(self.K, self.__argmin(roots, u).list())
        

In [411]:
rs1 = ReedSolomon(8, 6, 4)
print(rs1.n-rs1.t)

5


In [165]:
channel_t_errors(vector([rs1.g, rs1.g^1, 0, rs1.g^6]), 2)

(g, g, 1, g^2 + g)

In [166]:
RS = VectorSpace(rs1.K, rs1.k)
wmin = rs1.n
for m in RS:
    w_tmp = rs1.encode(m).hamming_weight()
    if w_tmp > 0 and w_tmp < wmin:
        wmin = w_tmp
print(wmin)

3


In [412]:
m = VectorSpace(rs1.K, rs1.k).random_element()

print("m = ", m)

m_enc = rs1.encode(m)
print("RS_encode(m) = ", m_enc)

# la distance minimale est 3 donc on peut corriger au max 1 erreur
noisy = channel_t_errors(m_enc, rs1.t)
print("Encoded + noise = ", noisy)

dec = rs1.BerlekampWelch(noisy)
print("Decoded = ", dec)
print("Success = ", m == dec)

m =  (g^2 + g, g^2 + g + 1, 0, g + 1)
RS_encode(m) =  (g, g^2 + g, g + 1, g + 1, 0, g + 1)
Encoded + noise =  (g, g^2 + g, g^2, g + 1, 0, g + 1)
Decoded =  (g^2 + g, g^2 + g + 1, 0, g + 1)
Success =  True


In [413]:
def vec_eq(x, y):
    for i in range(x.length()):
        if x[i] != y[i]:
            return False
    return True

m = VectorSpace(rs1.K, rs1.k).random_element()

print("m = ", m)

m_enc = rs1.encode(m)
print("RS_encode(m) = ", m_enc)


noisy = channel_t_errors(m_enc, rs1.t)
print("Encoded + noise = ", noisy)
   
dec = rs1.Sudan(noisy, rs1.t, 10)
print("Decoded = ", dec)
print("Success = ", m == dec)

m =  (0, g + 1, g^2, g + 1)
RS_encode(m) =  (g^2, g^2 + 1, g^2 + g, 0, 0, g + 1)
Encoded + noise =  (g^2, g^2 + 1, 1, 0, 0, g + 1)
u= (g^2, g^2 + 1, 1, 0, 0, g + 1)
M= [          1           1           1           1           1         g^2         g^2]
[          1           g         g^2       g + 1     g^2 + g     g^2 + 1           1]
[          1         g^2     g^2 + g     g^2 + 1           g           1         g^2]
[          1       g + 1     g^2 + 1         g^2 g^2 + g + 1           0           0]
[          1     g^2 + g           g g^2 + g + 1         g^2           0           0]
[          1 g^2 + g + 1       g + 1           g     g^2 + 1       g + 1           g]
coeffs= (0, 1, g, g^2 + g + 1, g^2 + g + 1, g^2 + g, g^2)
Q(Y)= (g^2*X + g^2 + g)*Y + (g^2 + g + 1)*X^4 + (g^2 + g + 1)*X^3 + g*X^2 + X
Decoded =  (0, g + 1, g^2, g + 1)
Success =  True


In [415]:

m = VectorSpace(rs1.K, rs1.k).random_element()

print("m = ", m)

m_enc = rs1.encode(m)
print("RS_encode(m) = ", m_enc)


noisy = channel_t_errors(m_enc, rs1.t)
print("Encoded + noise = ", noisy)
   
dec = rs1.GuruswamiSudan(noisy, 2, 5, rs1.t)
print("Decoded = ", dec)
print("Success = ", m == dec)

m =  (g^2 + g, g, g + 1, g)
RS_encode(m) =  (g^2 + 1, g + 1, g^2 + 1, g^2 + g + 1, g, g)
Encoded + noise =  (g^2 + 1, g + 1, g^2 + 1, g^2 + g + 1, g, g^2 + g)
M= [          1           1           1           1           1           1           1           1           1           1     g^2 + 1     g^2 + 1     g^2 + 1     g^2 + 1     g^2 + 1     g^2 + 1     g^2 + 1 g^2 + g + 1 g^2 + g + 1 g^2 + g + 1 g^2 + g + 1     g^2 + g]
[          0           0           0           0           0           0           0           0           0           0           1           1           1           1           1           1           1           0           0           0           0 g^2 + g + 1]
[          0           1           0           1           0           1           0           1           0           1           0     g^2 + 1           0     g^2 + 1           0     g^2 + 1           0 g^2 + g + 1           0 g^2 + g + 1           0     g^2 + g]
[          1           g         g^2    